In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']="5" 

In [2]:
import os
import gc

import math
import copy
import time
import random
import glob
from matplotlib import pyplot as plt
import h5py
# For data manipulation
import numpy as np
import pandas as pd
from PIL import Image
from io import BytesIO
# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp

from sklearn.metrics import roc_curve, auc, roc_auc_score
# Utils
import joblib
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, StratifiedGroupKFold 

# For Image Models
import timm
from glob import glob
# Albumentations for augmentations
#import albumentations as A
#from albumentations.pytorch import ToTensorV2
from timm.data.mixup import Mixup
from timm.loss import SoftTargetCrossEntropy
# For colored terminal text
from libauc.sampler import DualSampler  # data resampling (for binary class)
from torchvision import datasets, transforms
import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [3]:
CONFIG = {
    "seed": 42,
    "epochs": 10,
    "img_size": 384,
    "model_name": "resnet18",
    "checkpoint_path" : "./resnet18_pretrained/resnet18.pth",
    "train_batch_size": 64,
    "valid_batch_size": 64,
    "learning_rate": 5e-4,
    "scheduler": "CosineAnnealingLR",
    "min_lr": 1e-6,
    "T_max": 500,
    "weight_decay": 0,
    "fold" : 0,
    "n_fold": 20,
    "n_accumulate": 1,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    'output_dir': 'resnet18_pretrained'
}

In [4]:
os.makedirs(CONFIG['output_dir'], exist_ok=True)

In [5]:
url = timm.models.get_pretrained_cfg('resnet18').url
print(f"Pretrained model URL: {url}")

Pretrained model URL: https://github.com/huggingface/pytorch-image-models/releases/download/v0.1-rsb-weights/resnet18_a1_0-d63eafa0.pth


# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Set Seed for Reproducibility</h1></span>

In [6]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(CONFIG['seed'])

In [7]:
df_2024 = pd.read_csv(f"./2024/train-metadata.csv")
df_2024['file_path'] = df_2024['isic_id'].apply(lambda x:f'./2024/train-image/image/{x}.jpg')
df_2024 = df_2024[['patient_id', 'isic_id', 'target', 'file_path']]

In [8]:
df_2020 = pd.read_csv("./2020/train-metadata.csv")
df_2020['file_path'] = df_2020['isic_id'].apply(lambda x:f'./2020/train-image/image/{x}.jpg')
df_2020 = df_2020[['patient_id', 'isic_id', 'target', 'file_path']]

df_2019 = pd.read_csv("./2019/train-metadata.csv")
df_2019['file_path'] = df_2019['isic_id'].apply(lambda x:f'./2019/train-image/image/{x}.jpg')
df_2019 = df_2019[['patient_id', 'isic_id', 'target', 'file_path']]

df_2018 = pd.read_csv("./2018/train-metadata.csv")
images_path = glob('./2018/train-image/image/*.jpg')
df_2018['file_path'] = df_2018['isic_id'].apply(lambda x:f'./2018/train-image/image/{x}.jpg')
df_2018 = df_2018[['patient_id', 'isic_id', 'target', 'file_path']]
df_2018 = df_2018[df_2018['file_path'].isin(images_path)]

In [9]:
df = pd.concat([df_2020, df_2019, df_2018]).reset_index(drop=True)

In [10]:
len(df[df['target']==1])

5768

In [11]:
len(df[df['target']==0])

60185

In [12]:
CONFIG['T_max'] = df.shape[0] * (CONFIG["n_fold"]-1) * CONFIG['epochs'] // CONFIG['train_batch_size'] // CONFIG["n_fold"]
CONFIG['T_max']

9789

In [13]:
sgkf = StratifiedGroupKFold(n_splits=CONFIG['n_fold'])

for fold, ( _, val_) in enumerate(sgkf.split(df, df.target,df.patient_id)):
      df.loc[val_ , "kfold"] = int(fold)

In [14]:
def make_valid_augment(image_size):
    transform = transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),
        #transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ])
    return transform




def make_train_augment(image_size):
    augment_policy = 'autoaugment'  # 可以替换为其他策略，如 'autoaugment', 'trivialaugment'

    # 创建数据增强变换
    transform = transforms.Compose([
        transforms.Resize(image_size),
        transforms.RandomCrop((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        #transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ])
    return transform


class SkinDataset(Dataset):
    def __init__(self, df, augment):
        self.df = df
        self.targets = df['target'].tolist()
        self.augment = augment
        self.length = len(self.df)

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        d = self.df.iloc[index]
        img_path = d.file_path
        m = Image.open(img_path)
        r = self.augment(m)
       

        r = {
            #'index': index,
            'image': r,
            'target':self.targets[index]
        }
        return r

def null_collate(batch):
    d = {}
    key = batch[0].keys()
    for k in key:
        d[k] = [b[k] for b in batch]
    d['image'] = torch.stack(d['image']).float()
    d['target'] = torch.as_tensor(d['target'], dtype=torch.long)#.float()

    return d

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Augmentations</h1></span>

In [15]:
def F_rgb2hsv(rgb: torch.Tensor) -> torch.Tensor:
    cmax, cmax_idx = torch.max(rgb, dim=1, keepdim=True)
    cmin = torch.min(rgb, dim=1, keepdim=True)[0]
    delta = cmax - cmin
    hsv_h = torch.empty_like(rgb[:, 0:1, :, :])
    cmax_idx[delta == 0] = 3
    hsv_h[cmax_idx == 0] = (((rgb[:, 1:2] - rgb[:, 2:3]) / delta) % 6)[cmax_idx == 0]
    hsv_h[cmax_idx == 1] = (((rgb[:, 2:3] - rgb[:, 0:1]) / delta) + 2)[cmax_idx == 1]
    hsv_h[cmax_idx == 2] = (((rgb[:, 0:1] - rgb[:, 1:2]) / delta) + 4)[cmax_idx == 2]
    hsv_h[cmax_idx == 3] = 0.
    hsv_h /= 6.
    hsv_s = torch.where(cmax == 0, torch.tensor(0.).type_as(rgb), delta / cmax)
    hsv_v = cmax
    return torch.cat([hsv_h, hsv_s, hsv_v], dim=1)


In [16]:
class Net(nn.Module):
    def __init__(self, pretrained=False):
        super(Net, self).__init__()
        self.output_type = ['infer', 'loss']
        arch = CONFIG['model_name']

        self.model_name = CONFIG['model_name']

        self.model = timm.create_model(model_name=arch, pretrained=pretrained, in_chans=6,  num_classes=0, global_pool='')
        #print(f'Feature channels: {self.model.feature_info}')
        in_features =  self.model.num_features
        self.target=nn.Linear(in_features, 2)

        self.dropout = nn.ModuleList([
            nn.Dropout(0.5) for i in range(5)
        ]) #droupout augmentation

        self.loss_fn = SoftTargetCrossEntropy().cuda()
    def forward(self, batch):
        image = batch['image']
        batch_size = len(image)
        image = image.float()/255
        image = torch.cat([image, F_rgb2hsv(image)],1)
        # image = (image-self.mean)/self.std
        x = self.model(image)
        pool = F.adaptive_avg_pool2d(x,1).reshape(batch_size,-1)

        if self.training:
            logit=0
            for i in range(len(self.dropout)):
                logit += self.target(self.dropout[i](pool))
            logit = logit/len(self.dropout)
        else:
            logit = self.target(pool)
        output = {}
        target = batch['target']
        
        #output['loss'] = self.loss_fn(logit, target)
        pred_prob = logit.softmax(dim=-1)[:, 1]
        #print(pred_prob)
        output['logit'] = logit
        output['pred'] = pred_prob
        output['pool'] = pool
        return output


In [17]:
#------------------------------------------------------------------------
def run_check_net():
	image_size = 256
	batch_size = 32

	batch = {
		'image': torch.from_numpy(np.random.uniform(-1,1, (batch_size, 3, image_size, image_size))).float(),#.cuda(),
		'target': torch.from_numpy(np.random.choice(2, (batch_size,1))).long(),#.cuda(),
	}

	net = Net(pretrained=False)#.cuda()
	#print(net)

	with torch.no_grad():
		with torch.cuda.amp.autocast(enabled=True):
			output = net(batch)
	# ---
	print('batch')
	for k, v in batch.items():
		print(f'{k:>32} : {v.shape} ')

	print('output')
	for k, v in output.items():
		if 'loss' not in k:
			print(f'{k:>32} : {v.shape} ')
	print('loss')
	for k, v in output.items():
		if 'loss' in k:
			print(f'{k:>32} : {v.item()} ')

run_check_net()
print('model ok!')            

batch
                           image : torch.Size([32, 3, 256, 256]) 
                          target : torch.Size([32, 1]) 
output
                           logit : torch.Size([32, 2]) 
                            pred : torch.Size([32]) 
                            pool : torch.Size([32, 512]) 
loss
model ok!


In [18]:
def comp_score(gts, preds, min_tpr: float=0.80):
    v_gt = abs(np.asarray(gts)-1)
    v_pred = np.array([1.0 - x for x in preds])
    max_fpr = abs(1-min_tpr)
    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    # change scale from [0.5, 1.0] to [0.5 * max_fpr**2, max_fpr]
    # https://math.stackexchange.com/questions/914823/shift-numbers-into-a-different-range
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    return partial_auc

In [19]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch, mixup_fn=None):
    model.train()
    
    dataset_size = 0
    running_loss = 0.0
    running_auroc  = 0.0
    if mixup_fn:
        loss_fn = SoftTargetCrossEntropy().cuda()
    else:
        loss_fn = nn.CrossEntropyLoss().cuda()
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    preds = []
    gts = []
    for step, data in bar:
        for k, v in data.items():
            data[k] = v.to(device, dtype=torch.float)
        
        batch_size = data['image'].size(0)
        if mixup_fn:
            image = data['image']
            ori_target = data['target']
            
            image, target = mixup_fn(image, ori_target)
            data['image'] = image
            data['target'] = target

        outputs = model(data)
        if mixup_fn:
            loss = loss_fn(outputs['logit'], data['target'])
            targets = ori_target
        else:
            loss = F.cross_entropy(outputs['logit'],  data['target'].long())
            targets = data['target'].long()
        pred = outputs['pred']
        
        gts.append(targets.squeeze().cpu().numpy().reshape(-1))
        preds.append(pred.squeeze().detach().cpu().numpy().reshape(-1))
        loss = loss / CONFIG['n_accumulate']
            
        loss.backward()
    
        if (step + 1) % CONFIG['n_accumulate'] == 0:
            optimizer.step()

            # zero the parameter gradients
            optimizer.zero_grad()

            if scheduler is not None:
                scheduler.step()
                
        #auroc = comp_score(targets.squeeze().cpu().numpy(), preds.squeeze().detach().cpu().numpy())
        
        running_loss += (loss.item() * batch_size)
        #running_auroc  += (auroc * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        #epoch_auroc = running_auroc / dataset_size
        
        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss,)
    gc.collect()
    gts = np.concatenate(gts)
    preds = np.concatenate(preds)
    epoch_auroc = comp_score(gts, preds)
    return epoch_loss, epoch_auroc

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Validation Function</h1></span>

In [20]:
@torch.inference_mode()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    running_auroc = 0.0
    preds = []
    gts = []
    pools = []
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:        
        for k, v in data.items():
            data[k] = v.to(device, dtype=torch.float)
        
        batch_size = data['image'].size(0)
        
        outputs = model(data)
        loss = F.cross_entropy(outputs['logit'],  data['target'].long())
        pred = outputs['pred']
        targets = data['target']
        pool = outputs['pool']
        gts.append(targets.squeeze().cpu().numpy().reshape(-1))
        preds.append(pred.squeeze().detach().cpu().numpy().reshape(-1))
        pools.append(pool.squeeze().detach().cpu().numpy())
        #auroc = comp_score(targets.squeeze().cpu().numpy(), preds.squeeze().detach().cpu().numpy())
        running_loss += (loss.item() * batch_size)
        #running_auroc  += (auroc * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        #epoch_auroc = running_auroc / dataset_size
        
        bar.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss)   
    
    gc.collect()
    gts = np.concatenate(gts)
    preds = np.concatenate(preds)
    epoch_auroc = comp_score(gts, preds)
    pools = np.vstack(pools)
    return epoch_loss, epoch_auroc, pools, preds

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Run Training</h1></span>

In [29]:
def run_training(model, optimizer, scheduler, device, num_epochs, fold):
    if torch.cuda.is_available():
        print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))
    
    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_auroc = -np.inf
    history = defaultdict(list)
    mixup_args = {
                'mixup_alpha': 0.5,
                'cutmix_alpha': 0.5,
                'cutmix_minmax': None,
                'prob': 1.0,
                'switch_prob': 0.5,
                'mode': 'elem',
                'label_smoothing': 0,
                'num_classes': 2}
    mixup_fn = None #Mixup(**mixup_args)
    for epoch in range(1, num_epochs + 1): 
        gc.collect()
        train_epoch_loss, train_epoch_auroc = train_one_epoch(model, optimizer, scheduler, 
                                           dataloader=train_loader, 
                                           device=CONFIG['device'], epoch=epoch, mixup_fn=mixup_fn)
        
        val_epoch_loss, val_epoch_auroc, val_features, val_preds = valid_one_epoch(model, valid_loader, device=CONFIG['device'], 
                                         epoch=epoch)
    
        history['Train Loss'].append(train_epoch_loss)
        history['Valid Loss'].append(val_epoch_loss)
        history['Train AUROC'].append(train_epoch_auroc)
        history['Valid AUROC'].append(val_epoch_auroc)
        #history['lr'].append( scheduler.get_lr()[0] )
        
        # deep copy the model
        if best_epoch_auroc <= val_epoch_auroc:
            print(f"Validation AUROC Improved ({best_epoch_auroc} ---> {val_epoch_auroc})")
            best_epoch_auroc = val_epoch_auroc
            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = "AUROC{:.4f}_Loss{:.4f}_epoch{:.0f}.bin".format(val_epoch_auroc, val_epoch_loss, epoch)
            output_dir = CONFIG['output_dir']
            # Save a model file from the current directory
            print(f"Model Saved")
            best_val_features = val_features
            best_val_preds = val_preds
        print()
        torch.save(model.state_dict(), f'./{output_dir}/fold_{fold}_best.bin')
    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best AUROC: {:.4f}".format(best_epoch_auroc))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, history, best_val_features, best_val_preds

In [30]:
def fetch_scheduler(optimizer):
    if CONFIG['scheduler'] == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=CONFIG['T_max'], 
                                                   eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=CONFIG['T_0'], 
                                                             eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == None:
        return None
        
    return scheduler

In [31]:
def prepare_loaders(df, fold):
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    
    train_dataset = SkinDataset(df_train, augment=make_train_augment(image_size=CONFIG['img_size']))
    valid_dataset = SkinDataset(df_valid, augment=make_valid_augment(image_size=CONFIG['img_size']))
    sampler = DualSampler(train_dataset, batch_size=CONFIG['train_batch_size'], sampling_rate=0.1)

    train_loader = DataLoader(train_dataset, batch_size=CONFIG['train_batch_size'], 
                              num_workers=2, shuffle=False, pin_memory=True, collate_fn=null_collate,
                             sampler=sampler)
    valid_loader = DataLoader(valid_dataset, batch_size=CONFIG['valid_batch_size'], 
                              num_workers=2, shuffle=False, pin_memory=True, collate_fn=null_collate,)
    
    return train_loader, valid_loader

In [32]:
oof = []
for i in range(1):
    CONFIG["fold"] = i
    train_loader, valid_loader = prepare_loaders(df, fold=CONFIG["fold"])
    model = Net(False)
    pre_weight =torch.load(CONFIG['checkpoint_path'])
    model.load_state_dict(pre_weight, strict=False)
    model.cuda()
    optimizer = optim.AdamW(model.parameters(), lr=CONFIG['learning_rate'], 
                           weight_decay=CONFIG['weight_decay'])
    scheduler = fetch_scheduler(optimizer)
    model, history, best_val_features, best_val_preds = run_training(model, optimizer, scheduler,
                                  device=CONFIG['device'],
                                  num_epochs=CONFIG['epochs'],
                                  fold=CONFIG["fold"]
                                 )
    
    val_df = df[df.kfold == fold].reset_index(drop=True)[['patient_id', 'isic_id']]
    val_features =  pd.DataFrame(best_val_features, columns=[f'cnn_features_{i}' for i in range(best_val_features.shape[1])])
    val_df = pd.concat([val_df, val_features], axis=1)
    val_df['preds'] = best_val_preds
    oof.append(val_df)
oof = pd.concat(oof)

[INFO] Using GPU: NVIDIA GeForce RTX 4090



100%|██████████| 52/52 [00:05<00:00,  8.72it/s, Epoch=1, Valid_Loss=0.25]  


Validation AUROC Improved (-inf ---> 0.10667030051713368)
Model Saved



100%|██████████| 52/52 [00:05<00:00,  8.71it/s, Epoch=2, Valid_Loss=0.258] 


Validation AUROC Improved (0.10667030051713368 ---> 0.11398215963413104)
Model Saved



100%|██████████| 52/52 [00:05<00:00,  9.08it/s, Epoch=4, Valid_Loss=0.227] 


Validation AUROC Improved (0.115064035114955 ---> 0.12439038133580801)
Model Saved



100%|██████████| 52/52 [00:05<00:00,  8.85it/s, Epoch=5, Valid_Loss=0.229] 


 38%|███▊      | 375/985 [00:32<00:52, 11.64it/s, Epoch=6, Train_Loss=0.229]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

